# Exploring Graphs of Even and Odd Paths
-----

## Table of Contents

* [Model Setup](#Model-Setup)
    * [Model Introduction](#Model-Introduction)
    * [Dependencies](#Dependencies)
* [Model Design](#Model-Design)
    * [Random Graph Generator](#Random-Graph-Generator)
    * [Define the Genetic Algorithm Skeleton](#Define-the-Genetic-Algorithm-Skeleton)
    * [Automation: Defining the Simulation Suite](#Automation:-Defining-the-Graph-Simulation-Suite)
* [Simulations](#Simulations)
    * [Odd Paths](#Odd-Paths)
    * [Even Paths](#Even-Paths)
* [Conclusions](#Conclusions)
    * [Odd Path Review](#Odd-Paths-Review)
    * [Even Path Review](#Even-Paths-Review)

---
# Model Setup
----

## Model Introduction

In order to build a decent pool of data for us to collect information and derive possible schemas for coloring, investigating the differences between Odd and Even paths is critical. We will first analyze Odd paths and then even and wrap it all up in the [conclusions](#Conclusions) section.

## Dependencies

Grouping our simulation dependencies together for neatness and organization.

In [2]:
import sys
sys.path.append("../..")

from classes.genetic_algorithm import GeneticAlgorithm
from graph_coloring.classes.gc_ruleset import GCRuleset
from graph_coloring.classes.gc_random_init_strategy import GCRandomInitStrategy

----
# Model Design
---

In order for these simulations to be run in an automated fashion, let's define a factory function to build out graphs based on a given number of vertices.

## Random Graph Generator

In [1]:
def generate_path(num_vertices: int):
    """
    Generates a path with the provided number of vertices
    
    Parameters:
        num_vertices: the number of vertices in the path
    
    Returns:
        A path of a specified length
    
    """
    path = []
    
    for v in range(num_vertices):
        
        prev_vtx = 0 if v == 0 else v - 1
        
        next_vtx = (v + 1) % num_vertices
        
        path.append({"color": 0, "adj": [prev_vtx, next_vtx]})
    return path

In [37]:
generate_path(5)

[{'color': 0, 'adj': [0, 1]},
 {'color': 0, 'adj': [0, 2]},
 {'color': 0, 'adj': [1, 3]},
 {'color': 0, 'adj': [2, 4]},
 {'color': 0, 'adj': [3, 0]}]

Now let's define our Genetic Algorithm object, where the parameters are:

    - `ruleset` - The current ruleset being tested
    - `random_on_init_strategy`
    - `strat_data`

The last two will determine how each member of the population gets their respective strategey. The only two major aspects we are concerned with for this simulation are the *number of vertices in the path* and the *number of colors* ($k$) in the game.

## Define the Genetic Algorithm Skeleton

In [38]:
def path_test(num_vertices: int, k_colors: int):
    """
    Performs an evolutionary algorithm on a Path graph with the provided number of vertices and colors.
    
    Parameters:
        num_vertices: Order of the path
        k_colors: Number of colors for this game
        
    Returns:
        Two populations of players after the evolution
    """
    # Define the Path
    initial_state = generate_path(num_vertices)
    
    # Define the Ruleset of the game
    ruleset = GCRuleset("Graph Coloring Ruleset", initial_state, bounds = k_colors)
    
    # Create a new Evolution instance with the example strategy
    gen_algo = GeneticAlgorithm(
        ruleset,
        # Random-on-initialization strategy for generating populations of random players
        random_on_init_strat = GCRandomInitStrategy,
        # Data to be used by the above strategy
        strat_data = {"vertices": range(len(ruleset.initial_state)), "colors": range(1, ruleset.bounds + 1)},
        # Size of the populations
        pop_size = 100,
        # Number of generations to iterate through
        iterations = 10,
        # Minimum number of games each player must play during a generation
        num_games = 10,
        # Starting fitness threshold
        fitness = 0.5,
        # Maximum fitness threshold; be careful of setting this too close to 1.0
        max_fitness = 0.9,
        # How much the fitness threshold should increment after each iteration
        fitness_increment = 0.025,
        # Chance of a mutation to occur during player reproduction
        mutation_rate = 0.025
    )

    p1_pop, p2_pop = gen_algo.evolve(to_df=True)
    
    return p1_pop, p2_pop


## Automation: Defining the Graph Simulation Suite

In [2]:
def graph_simulation_suite(orders, k_colors, graph_test):
    """
    Runs a series of simulations on the orders and k_colors
    
    Parameters:
        orders: A list of graph orders to test
        k_colors: A list of colors that each game might have
        graph_test: A function that will take the params and run the simulations
    
    Returns:
        simulations: A dictionary containing an entry for each order and the respective 
                     k colorings winning player populations
    """
    simulations = {}
    for order in orders:
        current_run = {}
        print("----------------------------------------------------------------------")
        for k in k_colors:
            print("\n\tTesting Graph of Order " + str(order) + " with k = " + str(k))
            p1_pop, p2_pop = graph_test(num_vertices = order, k_colors = k)
            print(p1_pop.head())
            print(p2_pop.head())
            current_run[str(k)] = (p1_pop, p2_pop)
        simulations[str(order)] = current_run
        print("----------------------------------------------------------------------")
    return simulations


----
# Simulations
----

## Odd Paths

Now to run a series of experiments and prepare to analyze the data. Where the number of colors is $k = 3$.

In [53]:
p1_pop, p2_pop = path_test(num_vertices = 5, k_colors = 3)

In [54]:
p1_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
97,Player 1,0,"[0, 2, 3, 1, 4]","[2, 3, 1]",1.0,227,0
66,Player 1,0,"[0, 4, 1, 2, 3]","[2, 1, 3]",1.0,221,0
50,Player 1,0,"[3, 2, 1, 4, 0]","[3, 1, 2]",1.0,219,0
23,Player 1,0,"[4, 0, 2, 3, 1]","[3, 1, 2]",1.0,218,0
68,Player 1,0,"[3, 1, 0, 4, 2]","[3, 1, 2]",1.0,218,0


In [55]:
p2_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
0,Player 2,0,"[3, 1, 2, 4, 0]","[1, 2, 3]",0.0,0,0
1,Player 2,0,"[2, 3, 0, 1, 4]","[3, 2, 1]",0.0,0,0
2,Player 2,0,"[3, 4, 0, 2, 1]","[1, 2, 3]",0.0,0,0
3,Player 2,0,"[0, 1, 3, 2, 4]","[3, 2, 1]",0.0,0,0
4,Player 2,0,"[1, 4, 2, 3, 0]","[3, 2, 1]",0.0,0,0


The results are in and it looks like player 1 is winning 100% of the matches on a path of order 6. Let's try on a graph with order 9 and see how that affects the results.

In [45]:
p1_pop, p2_pop = path_test(num_vertices = 9, k_colors = 3)
print(p1_pop.head())
print(p2_pop.head())

        Name Gen                     Vertices     Colors  Fitness Wins Losses
73  Player 1   0  [3, 5, 4, 1, 8, 0, 6, 7, 2]  [3, 1, 2]      1.0  222      0
7   Player 1   0  [7, 5, 4, 8, 0, 2, 3, 1, 6]  [2, 1, 3]      1.0  220      0
18  Player 1   0  [6, 1, 2, 7, 5, 4, 0, 3, 8]  [2, 1, 3]      1.0  219      0
50  Player 1   0  [2, 6, 3, 8, 4, 0, 1, 5, 7]  [2, 1, 3]      1.0  217      0
23  Player 1   0  [0, 7, 4, 8, 1, 3, 6, 2, 5]  [3, 2, 1]      1.0  216      0
       Name Gen                     Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [3, 8, 5, 1, 2, 4, 7, 6, 0]  [3, 1, 2]      0.0    0      0
1  Player 2   0  [1, 8, 3, 0, 2, 5, 7, 4, 6]  [2, 3, 1]      0.0    0      0
2  Player 2   0  [3, 2, 1, 5, 8, 7, 4, 0, 6]  [2, 3, 1]      0.0    0      0
3  Player 2   0  [1, 4, 8, 0, 6, 2, 7, 5, 3]  [3, 1, 2]      0.0    0      0
4  Player 2   0  [5, 3, 2, 8, 0, 6, 4, 7, 1]  [1, 2, 3]      0.0    0      0


In one last attempt, lets try several large odd ordered graphs.

In [50]:
for order in [11, 33, 43]:
    print("\tTesting on cycle of order: " + str(order))
    p1_pop, p2_pop = path_test(num_vertices=order, k_colors=3)
    print(p1_pop.head())
    print(p2_pop.head())

	Testing on cycle of order: 11
        Name Gen                            Vertices     Colors  Fitness Wins  \
63  Player 1   0  [5, 7, 6, 8, 0, 1, 2, 10, 9, 4, 3]  [2, 3, 1]      1.0  222   
8   Player 1   0  [7, 6, 4, 9, 8, 2, 1, 0, 5, 3, 10]  [1, 2, 3]      1.0  218   
82  Player 1   0  [3, 9, 7, 10, 1, 6, 2, 5, 8, 0, 4]  [1, 2, 3]      1.0  218   
52  Player 1   0  [5, 4, 10, 6, 3, 9, 1, 2, 7, 8, 0]  [1, 2, 3]      1.0  217   
68  Player 1   0  [10, 8, 9, 6, 5, 4, 7, 3, 2, 0, 1]  [2, 1, 3]      1.0  216   

   Losses  
63      0  
8       0  
82      0  
52      0  
68      0  
       Name Gen                            Vertices     Colors  Fitness Wins  \
0  Player 2   0  [4, 1, 10, 2, 0, 6, 7, 5, 9, 3, 8]  [1, 3, 2]      0.0    0   
1  Player 2   0  [9, 5, 8, 3, 10, 6, 4, 1, 0, 2, 7]  [1, 2, 3]      0.0    0   
2  Player 2   0  [4, 2, 6, 0, 1, 7, 10, 5, 9, 3, 8]  [1, 3, 2]      0.0    0   
3  Player 2   0  [10, 4, 7, 2, 5, 3, 9, 6, 8, 0, 1]  [1, 2, 3]      0.0    0   
4  Player 

Alright it seems that on an odd path with $k = 3$ is in fact a winning chromatic number. Lets run a few scenarios where we shift that value. We will test on the same size graphs to keep it consistent.

In [63]:
orders = [11, 33, 45]
k_colors = [2,3,4,5]

In [74]:
simulation1 = graph_simulation_suite(orders, k_colors, path_test)

----------------------------------------------------------------------

	Testing Graph of Order 12 with k = 2
       Name Gen                                Vertices  Colors  Fitness Wins  \
0  Player 1   0  [6, 4, 0, 5, 1, 3, 8, 7, 2, 10, 9, 11]  [2, 1]      0.0    0   
1  Player 1   0  [7, 2, 3, 11, 9, 8, 5, 4, 6, 0, 10, 1]  [1, 2]      0.0    0   
2  Player 1   0  [7, 3, 9, 11, 10, 8, 6, 1, 2, 4, 0, 5]  [1, 2]      0.0    0   
3  Player 1   0  [5, 10, 11, 1, 6, 3, 2, 7, 9, 4, 8, 0]  [1, 2]      0.0    0   
4  Player 1   0  [2, 7, 0, 1, 8, 6, 4, 3, 9, 5, 10, 11]  [1, 2]      0.0    0   

  Losses  
0      0  
1      0  
2      0  
3      0  
4      0  
       Name Gen                                Vertices  Colors   Fitness  \
0  Player 2   2  [1, 2, 6, 4, 10, 7, 5, 0, 11, 8, 9, 3]  [2, 1]  0.964912   
1  Player 2   1  [1, 10, 6, 7, 0, 5, 3, 8, 2, 11, 9, 4]  [2, 1]  0.962500   
2  Player 2   1  [1, 2, 6, 4, 10, 11, 5, 0, 7, 8, 9, 3]  [2, 1]  0.962121   
3  Player 2   0  [11, 0, 4, 7

        Name Gen                                           Vertices  \
87  Player 1   0  [19, 3, 2, 14, 10, 6, 4, 1, 20, 15, 8, 9, 12, ...   
9   Player 1   0  [0, 7, 1, 11, 6, 18, 19, 23, 5, 8, 2, 16, 20, ...   
54  Player 1   0  [9, 14, 21, 3, 7, 5, 6, 16, 0, 18, 11, 23, 4, ...   
67  Player 1   0  [13, 23, 12, 18, 8, 22, 5, 0, 2, 6, 11, 4, 15,...   
95  Player 1   0  [18, 21, 9, 1, 19, 6, 23, 15, 8, 4, 17, 14, 11...   

             Colors  Fitness Wins Losses  
87  [4, 5, 3, 1, 2]      1.0  227      0  
9   [3, 4, 2, 1, 5]      1.0  224      0  
54  [2, 3, 5, 1, 4]      1.0  224      0  
67  [1, 5, 4, 2, 3]      1.0  219      0  
95  [3, 5, 1, 2, 4]      1.0  216      0  
       Name Gen                                           Vertices  \
0  Player 2   0  [14, 0, 20, 1, 17, 16, 4, 6, 12, 7, 2, 8, 3, 9...   
1  Player 2   0  [20, 6, 9, 18, 19, 4, 7, 3, 16, 8, 13, 14, 0, ...   
2  Player 2   0  [8, 19, 22, 6, 1, 14, 20, 23, 0, 3, 4, 12, 18,...   
3  Player 2   0  [7, 10, 23, 8, 18,

And now that we have collected all of those results in the variable `simulation1`, we can reference all the data frames later for further analysis. 

## Even Paths

Following the same process as for the odd path, we will now generate some data with regards to even paths. Let's start with $k = 3$ just like before and then we will move into the `graph_simulation_suite()`.

In [65]:
p1_pop, p2_pop = path_test(num_vertices = 4, k_colors = 3)

In [68]:
print(p1_pop.head())
print(p2_pop.head())

        Name Gen      Vertices     Colors  Fitness Wins Losses
26  Player 1   0  [0, 1, 2, 3]  [1, 2, 3]      1.0  231      0
64  Player 1   0  [2, 1, 3, 0]  [1, 3, 2]      1.0  226      0
24  Player 1   0  [3, 2, 1, 0]  [3, 1, 2]      1.0  221      0
59  Player 1   0  [2, 0, 3, 1]  [3, 2, 1]      1.0  218      0
87  Player 1   0  [3, 1, 0, 2]  [1, 3, 2]      1.0  218      0
       Name Gen      Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [1, 2, 0, 3]  [1, 3, 2]      0.0    0      0
1  Player 2   0  [2, 0, 3, 1]  [3, 1, 2]      0.0    0      0
2  Player 2   0  [0, 1, 2, 3]  [2, 3, 1]      0.0    0      0
3  Player 2   0  [3, 0, 2, 1]  [2, 3, 1]      0.0    0      0
4  Player 2   0  [1, 3, 2, 0]  [2, 1, 3]      0.0    0      0


Lets shorten the path a bit

In [72]:
p1_pop, p2_pop = path_test(num_vertices = 2, k_colors = 3)
print(p1_pop.head())
print(p2_pop.head())

Error: Failed to create unique players 1000 times. Aborting
Error: Failed to create unique players 1000 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
Error: Failed to create unique players 500 times. Aborting
        Name Gen Vertices     Colors  Fitness Wins Losses
4   Player 1   0   [1, 0]  [2, 1, 3]      1.0  219      0
60  Player 1   2   [0, 1]  [2, 3, 1]      1.0  215      0
10  Player 1   0   [0, 1]  [3, 1, 2]      1.0  210      0
0   Player 1   0   [0, 1]  [2, 1, 3]      1.0  208      0


An order of 2 (num_vertices = 2), appears to be a limitation of this current model and will be addressed in [Model Limitations](#Model-Limitations). 

Moving on to the test suite, defining the orders and k_colors to use in the model.

In [75]:
orders = [12, 24, 48]
k_colors = [2,3,4,5]
simulation2 = graph_simulation_suite(orders, k_colors, path_test);

----------------------------------------------------------------------

	Testing Graph of Order 12 with k = 2
       Name Gen                                Vertices  Colors  Fitness Wins  \
0  Player 1   0  [0, 10, 9, 11, 6, 7, 5, 1, 3, 8, 4, 2]  [2, 1]      0.0    0   
1  Player 1   0  [11, 0, 8, 2, 5, 1, 10, 7, 9, 4, 6, 3]  [2, 1]      0.0    0   
2  Player 1   0  [10, 11, 5, 3, 4, 9, 1, 7, 0, 8, 6, 2]  [2, 1]      0.0    0   
3  Player 1   0  [9, 10, 7, 11, 0, 5, 8, 1, 6, 4, 2, 3]  [1, 2]      0.0    0   
4  Player 1   0  [5, 10, 11, 6, 7, 3, 2, 4, 0, 1, 9, 8]  [1, 2]      0.0    0   

  Losses  
0      0  
1      0  
2      0  
3      0  
4      0  
       Name Gen                                Vertices  Colors   Fitness  \
0  Player 2   3  [10, 7, 1, 8, 5, 9, 2, 0, 3, 11, 6, 4]  [1, 2]  0.988764   
1  Player 2   0  [10, 5, 3, 2, 8, 7, 4, 6, 11, 0, 1, 9]  [2, 1]  0.960000   
2  Player 2   2  [8, 5, 6, 7, 4, 11, 3, 1, 2, 0, 9, 10]  [1, 2]  0.959350   
3  Player 2   2  [9, 4, 1, 6,

        Name Gen                                           Vertices  \
42  Player 1   0  [19, 10, 20, 17, 13, 6, 22, 11, 7, 4, 14, 18, ...   
2   Player 1   0  [17, 1, 5, 16, 21, 7, 13, 20, 18, 14, 4, 0, 11...   
35  Player 1   0  [14, 17, 4, 22, 3, 21, 6, 12, 18, 8, 20, 10, 1...   
77  Player 1   0  [6, 22, 1, 12, 8, 11, 7, 13, 21, 14, 16, 5, 9,...   
28  Player 1   0  [14, 21, 17, 20, 13, 3, 9, 4, 18, 10, 0, 2, 15...   

             Colors  Fitness Wins Losses  
42  [2, 1, 5, 4, 3]      1.0  222      0  
2   [1, 2, 3, 5, 4]      1.0  221      0  
35  [4, 2, 1, 3, 5]      1.0  220      0  
77  [1, 4, 2, 3, 5]      1.0  217      0  
28  [2, 4, 1, 5, 3]      1.0  216      0  
       Name Gen                                           Vertices  \
0  Player 2   0  [15, 16, 10, 6, 20, 1, 0, 23, 5, 22, 2, 9, 17,...   
1  Player 2   0  [18, 17, 11, 0, 14, 21, 13, 15, 9, 12, 23, 7, ...   
2  Player 2   0  [7, 0, 16, 22, 18, 13, 19, 12, 11, 4, 20, 14, ...   
3  Player 2   0  [8, 4, 18, 17, 12,

And now that we have collected all of those results in the variable `simulation2`, we can reference all the data frames later for further analysis. 

-----
# Model Limitations
----

Every model is bound to have limitations and the few that I noticed were:

- When $k = 2$ for the Even Paths. This caused our Genetic Algorithm to crash or would cause the player populations would struggle to create players 
- Large graphs require lots of computation power and would require that they be run on cloud computing systems such as our own cloud server or Google collab.

---
# Conclusions
---

## Odd Path Review

From viewing the data in `simulation1` we can see:

- $k = 2$ meant player 1 lost 100% of the time
- $k = 3$ meant player 1 won 100% of the time
- $k = 4$ meant player 1 won 100% of the time
- $k = 5$ meant player 1 won 100% of the time



## Even Paths Review

From viewing the data in `simulation2` we can see:

- $k = 2$ meant player 1 lost 100% of the time
- $k = 3$ meant player 1 won 100% of the time
- $k = 4$ meant player 1 won 100% of the time
- $k = 5$ meant player 1 won 100% of the time

Noting that the game chromatic coloring (Minimal coloring) is $k = 3$, we have successfully shown this to hold true.